In [1]:
import org.apache.spark.graphx._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.14:4047
SparkContext available as 'sc' (version = 3.0.3, master = local[*], app id = local-1642011063604)
SparkSession available as 'spark'


import org.apache.spark.graphx._


In [2]:
val myVertices = sc.makeRDD(Array((1L, "Ann"),
                                  (2L, "Bill"),
                                  (3L, "Charles"),
                                  (4L, "Diane"),
                                  (5L, "Went to gym this morning")))

val myEdges = sc.makeRDD(Array(Edge(1L, 2L, "is-friends-with"),
                               Edge(2L, 3L, "is-friends-with"), 
                               Edge(3L, 4L, "is-friends-with"),
                               Edge(4L, 5L, "Likes-status"), 
                               Edge(3L, 5L, "Wrote-status")))

val myGraph = Graph(myVertices, myEdges)

myGraph.vertices.collect

myVertices: org.apache.spark.rdd.RDD[(Long, String)] = ParallelCollectionRDD[0] at makeRDD at <console>:28
myEdges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[String]] = ParallelCollectionRDD[1] at makeRDD at <console>:34
myGraph: org.apache.spark.graphx.Graph[String,String] = org.apache.spark.graphx.impl.GraphImpl@286d7527
res0: Array[(org.apache.spark.graphx.VertexId, String)] = Array((1,Ann), (2,Bill), (3,Charles), (4,Diane), (5,Went to gym this morning))


In [3]:
myGraph.triplets.collect.foreach(println)

((1,Ann),(2,Bill),is-friends-with)
((2,Bill),(3,Charles),is-friends-with)
((3,Charles),(4,Diane),is-friends-with)
((4,Diane),(5,Went to gym this morning),Likes-status)
((3,Charles),(5,Went to gym this morning),Wrote-status)


In [4]:
myGraph.mapTriplets(t => (t.attr, t.attr=="is-friends-with" && 
                              t.srcAttr.toLowerCase.contains("a")))
                    .triplets.collect.foreach(println)

((1,Ann),(2,Bill),(is-friends-with,true))
((2,Bill),(3,Charles),(is-friends-with,false))
((3,Charles),(4,Diane),(is-friends-with,true))
((4,Diane),(5,Went to gym this morning),(Likes-status,false))
((3,Charles),(5,Went to gym this morning),(Wrote-status,false))


In [5]:
myGraph.mapTriplets((t => (t.attr, t.attr=="is-friends-with" && 
                               t.srcAttr.toLowerCase.contains("a"))) : 
                                    (EdgeTriplet[String,String] => Tuple2[String,Boolean]))
                    .triplets.collect.foreach(println)

((1,Ann),(2,Bill),(is-friends-with,true))
((2,Bill),(3,Charles),(is-friends-with,false))
((3,Charles),(4,Diane),(is-friends-with,true))
((4,Diane),(5,Went to gym this morning),(Likes-status,false))
((3,Charles),(5,Went to gym this morning),(Wrote-status,false))


In [6]:
myGraph.aggregateMessages[Int](_.sendToSrc(1), _ + _).collect.foreach(println)

(1,1)
(2,1)
(3,2)
(4,1)


In [7]:
myGraph.aggregateMessages[Int](_.sendToSrc(1), _ + _).join(myGraph.vertices)
    .collect.foreach(println)

(1,(1,Ann))
(2,(1,Bill))
(3,(2,Charles))
(4,(1,Diane))


In [8]:
myGraph.aggregateMessages[Int](_.sendToSrc(1), _ + _)
    .join(myGraph.vertices).map(_._2.swap).collect.foreach(println)

(Ann,1)
(Bill,1)
(Charles,2)
(Diane,1)


In [9]:
myGraph.aggregateMessages[Int](_.sendToSrc(1), _ + _)
    .rightOuterJoin(myGraph.vertices).map(_._2.swap).collect.foreach(println)

(Ann,Some(1))
(Bill,Some(1))
(Charles,Some(2))
(Diane,Some(1))
(Went to gym this morning,None)


In [10]:
myGraph.aggregateMessages[Int](_.sendToSrc(1), _ + _)
    .rightOuterJoin(myGraph.vertices).map(x => (x._2._2, x._2._1.getOrElse(0)))
    .collect.foreach(println)

(Ann,1)
(Bill,1)
(Charles,2)
(Diane,1)
(Went to gym this morning,0)


<hr>

Đoạn code này giúp tìm khoảng cách giữa 2 điểm xa nhất

In [11]:
def sendMsg(ec:EdgeContext[Int, String, Int]):Unit = {
    ec.sendToDst(ec.srcAttr + 1)
}

def mergeMsg(a:Int, b:Int):Int = {
    math.max(a, b)
}

def propagateEdgeCount(g:Graph[Int, String]):Graph[Int, String] = {
    val verts = g.aggregateMessages[Int](sendMsg, mergeMsg)
    val g2 = Graph(verts, g.edges)
    
    val check = g2.vertices.join(g.vertices)
                    .map(x => x._2._1 - x._2._2).reduce(_ + _)
    
    if (check > 0) propagateEdgeCount(g2)
    else g
}

sendMsg: (ec: org.apache.spark.graphx.EdgeContext[Int,String,Int])Unit
mergeMsg: (a: Int, b: Int)Int
propagateEdgeCount: (g: org.apache.spark.graphx.Graph[Int,String])org.apache.spark.graphx.Graph[Int,String]
